# Download Thumbail of youtube-video

In [1]:
import pandas as pd
import sqlite3
import requests
from tqdm import tqdm

In [2]:
db_file = 'smda/project/data/videos.db'

conn = sqlite3.connect(db_file)

query = '''
    Select *
    FROM videos
    WHERE view_count >= 10000
    ORDER BY RANDOM()
    LIMIT 50000
    '''
df = pd.read_sql_query(query, conn)

df.to_parquet('smda/project/data/random_50000_thumbnails.parquet', index=False)

In [4]:
# get thumbnail for each video

thumbnail_status = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    video_id = row['display_id']
    url = f'https://img.youtube.com/vi/{video_id}/hqdefault.jpg'
    response = requests.get(url)
    if response.status_code == 200:
        with open(f'smda/project/data/thumbnails/{video_id}.jpg', 'wb') as file:
            file.write(response.content)
        thumbnail_status.append('success')
    else:
        thumbnail_status.append('failed')

df['thumbnail_status'] = thumbnail_status

print(df['thumbnail_status'].value_counts())

100%|██████████| 50000/50000 [1:50:33<00:00,  7.54it/s]  

thumbnail_status
success    42080
failed      7920
Name: count, dtype: int64


In [5]:
df.to_parquet('smda/project/data/random_50000_thumbnails.parquet', index=False)